In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [2]:
media_df = pd.read_csv('media_data.csv')

In [3]:
media_df.columns

Index(['timedesc', 'year', 'month', 'count', 'model_region', 'regiondesc',
       'effect', 'prodhierarchy1', 'filter_brand_for_model', 'brand',
       'vehicle', 'campaign', 'subtype0', 'subtype1', 'subtype2',
       'model_vehicle', 'type', 'spend', 'currency', 'original_value', 'fact',
       'value', 'scaled_fact'],
      dtype='object')

In [4]:
# selected columns as it is channel level for now
media_df = media_df[['timedesc', 'effect','filter_brand_for_model', 'brand', 'vehicle', 'spend']]

In [5]:
media_df

,timedesc,effect,filter_brand_for_model,brand,vehicle,spend
0,2017-01-06,Target,CDC,CDC,TV,11350.0
1,2017-01-06,Target,CDC,CDC,TV,19125.0
2,2017-01-06,Target,CDC,CDC,TV,106000.0
3,2017-01-06,Target,CDC,CDC,TV,44850.0
4,2017-01-06,Target,CDC,CDC,TV,82537.5
...,...,...,...,...,...,...
32400,2020-12-25,Halo,CDC,EQUITY,TV,134787.5
32401,2020-12-25,Halo,CDC,EQUITY,TV,111225.0
32402,2020-12-25,Halo,CDC,EQUITY,TV,61625.0
32403,2020-12-25,Halo,CDC,EQUITY,TV,10175.0


In [6]:
media_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32405 entries, 0 to 32404
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timedesc                32405 non-null  object 
 1   effect                  32405 non-null  object 
 2   filter_brand_for_model  32405 non-null  object 
 3   brand                   32405 non-null  object 
 4   vehicle                 32405 non-null  object 
 5   spend                   32405 non-null  float64
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


In [7]:
# %Y	4-digit year	2017
# %y	2-digit year	17
# %m	Month (01–12)	01
# %d	Day of month (01–31)	06
# %H	Hour (00–23)	14
# %M	Minute (00–59)	30
# %S	Second (00–59)	45

In [8]:
# change format of timedesc to %Y-%m-%d
media_df['timedesc'] = pd.to_datetime(media_df['timedesc'], format="%Y-%m-%d")

In [9]:
media_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32405 entries, 0 to 32404
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   timedesc                32405 non-null  datetime64[ns]
 1   effect                  32405 non-null  object        
 2   filter_brand_for_model  32405 non-null  object        
 3   brand                   32405 non-null  object        
 4   vehicle                 32405 non-null  object        
 5   spend                   32405 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 1.5+ MB


In [10]:
media_df['filter_brand_for_model'].unique()

array(['CDC'], dtype=object)

In [11]:
# filter CDC as we have to do modeling for CDC
media_df = media_df[media_df['filter_brand_for_model'] == 'CDC']

In [12]:
# concat brand and vehicle
media_df['brand_vehicle'] = media_df['brand'] + '+' + media_df['vehicle']

In [13]:
media_df['brand_vehicle']

0           CDC+TV
1           CDC+TV
2           CDC+TV
3           CDC+TV
4           CDC+TV
           ...    
32400    EQUITY+TV
32401    EQUITY+TV
32402    EQUITY+TV
32403    EQUITY+TV
32404    EQUITY+TV
Name: brand_vehicle, Length: 32405, dtype: object

In [14]:
media_df

,timedesc,effect,filter_brand_for_model,brand,vehicle,spend,brand_vehicle
0,2017-01-06,Target,CDC,CDC,TV,11350.0,CDC+TV
1,2017-01-06,Target,CDC,CDC,TV,19125.0,CDC+TV
2,2017-01-06,Target,CDC,CDC,TV,106000.0,CDC+TV
3,2017-01-06,Target,CDC,CDC,TV,44850.0,CDC+TV
4,2017-01-06,Target,CDC,CDC,TV,82537.5,CDC+TV
...,...,...,...,...,...,...,...
32400,2020-12-25,Halo,CDC,EQUITY,TV,134787.5,EQUITY+TV
32401,2020-12-25,Halo,CDC,EQUITY,TV,111225.0,EQUITY+TV
32402,2020-12-25,Halo,CDC,EQUITY,TV,61625.0,EQUITY+TV
32403,2020-12-25,Halo,CDC,EQUITY,TV,10175.0,EQUITY+TV


In [15]:
# sum of spend for each brand_vehicle grouped by timedesc
media_data = media_df.groupby(['timedesc','brand_vehicle'])['spend'].sum().reset_index()

In [16]:
media_data

,timedesc,brand_vehicle,spend
0,2017-01-06,CDC+TV,1.319312e+06
1,2017-01-06,EQUITY+DIGITAL,1.291382e+05
2,2017-01-06,SALT+DIGITAL,9.317750e+01
3,2017-01-06,SALT+TV,1.780500e+06
4,2017-01-06,TOTAL+DIGITAL,1.609650e+02
...,...,...,...
1746,2020-12-25,PANJAVED+TV,2.960888e+06
1747,2020-12-25,SALT+DIGITAL,7.724475e+00
1748,2020-12-25,SALT+OTHERADVERTISING_NEWSPAPER_RADIO,2.750000e+05
1749,2020-12-25,TOTAL+DIGITAL,3.970304e+05


In [17]:
# transpose data into brand_vehicle as columns and spend as values
media_mrd = pd.pivot_table(media_data, values='spend', columns='brand_vehicle', index='timedesc', aggfunc=np.sum).reset_index()

C:\Users\Admin\AppData\Local\Temp\ipykernel_25964\862413336.py:2: FutureWarning: The provided callable <function sum at 0x000001D3B162D800> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  media_mrd = pd.pivot_table(media_data, values='spend', columns='brand_vehicle', index='timedesc', aggfunc=np.sum).reset_index()


In [18]:
media_mrd = media_mrd.fillna(0)

In [19]:
media_mrd = media_mrd.rename(columns={'timedesc':'week_date'})

In [20]:
media_mrd.to_excel('media_mrd.xlsx', index=False)

In [20]:
control_df = pd.read_csv('control_data.csv')

In [21]:
control_df.columns

Index(['timedesc', 'year', 'month', 'qaurter', 'channel', 'channel_yr',
       'channel_mo', 'channel_qr', 'channeldesc', 'prodhierarchy1',
       'prodhierarchy2', 'proddesc', 'regiondesc', 'region_year', 'value',
       'volume', 'nd', 'wd', 'channel_year', 'prod_year', 'aided_awareness',
       'closeness_top_3_box', 'loyalty', 'top_of_mind_brand_awareness',
       'total_unaided_brand_awareness', 'chim_shop_chai_',
       'panic_pantry_loading', 'welfare', 'welfare_additional_boost',
       'bigpack', 'buyget', 'extravol', 'free_promo', 'free_with_others',
       'freepremium', 'near_pack', 'pmd', 'rl', 'theme', 'tpr', 'target_price',
       'target_baseprice', 'target_discount', 'darlie_discount',
       'darlie_cdc_basepriceratio', 'week5_si', 'week13_si', 'trendline',
       'consumer_confidence_index', 'tourist_arrivals_in_million',
       'extravol_darlie', 'free_promo_darlie', 'free_with_others_darlie',
       'tpr_darlie'],
      dtype='object')

In [22]:
# format timedesc into %Y-%m-%d
control_df['timedesc'] = pd.to_datetime(control_df['timedesc'], format="%Y-%m-%d")

In [23]:
# start time of week date
start_date = media_mrd['week_date'].min()

In [25]:
start_date

Timestamp('2017-01-06 00:00:00')

In [24]:
# end time of week date
end_date = media_mrd['week_date'].max()

In [26]:
end_date

Timestamp('2020-12-25 00:00:00')

In [27]:
# all dates between start and end date
control_df = control_df[(control_df['timedesc'] >= start_date) & (control_df['timedesc'] <= end_date)]

In [28]:
media_mrd.head()

brand_vehicle,week_date,CDC+DIGITAL,CDC+TV,EQUITY+DIGITAL,EQUITY+IN STORE,EQUITY+OOH,EQUITY+OTHERADVERTISING_NEWSPAPER_RADIO,EQUITY+TV,PANJAVED+DIGITAL,PANJAVED+IN STORE,PANJAVED+OOH,PANJAVED+TV,SALT+DIGITAL,SALT+IN STORE,SALT+OTHERADVERTISING_NEWSPAPER_RADIO,SALT+TV,TOTAL+DIGITAL,TOTAL+IN STORE,TOTAL+OTHERADVERTISING_NEWSPAPER_RADIO,TOTAL+TV
0,2017-01-06,0.0,1319312.5,129138.2050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.1775,0.0,0.0,1780500.00,160.965,0.0,0.0,0.0
1,2017-01-13,0.0,1319312.5,129138.2050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.1775,0.0,0.0,1780500.00,160.965,0.0,0.0,0.0
2,2017-01-20,0.0,1319312.5,129138.2050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.1775,0.0,0.0,1780500.00,160.965,0.0,0.0,0.0
3,2017-01-27,0.0,1319312.5,129138.2050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.1775,0.0,0.0,1780500.00,160.965,0.0,0.0,0.0
4,2017-02-03,0.0,1318312.5,102321.6725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85103.8925,0.0,101800.0,2410031.25,158234.765,200050.0,0.0,1609500.0


In [29]:
list(media_mrd['week_date'].unique())

[Timestamp('2017-01-06 00:00:00'),
 Timestamp('2017-01-13 00:00:00'),
 Timestamp('2017-01-20 00:00:00'),
 Timestamp('2017-01-27 00:00:00'),
 Timestamp('2017-02-03 00:00:00'),
 Timestamp('2017-02-10 00:00:00'),
 Timestamp('2017-02-17 00:00:00'),
 Timestamp('2017-02-24 00:00:00'),
 Timestamp('2017-03-03 00:00:00'),
 Timestamp('2017-03-10 00:00:00'),
 Timestamp('2017-03-17 00:00:00'),
 Timestamp('2017-03-24 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-04-07 00:00:00'),
 Timestamp('2017-04-14 00:00:00'),
 Timestamp('2017-04-21 00:00:00'),
 Timestamp('2017-04-28 00:00:00'),
 Timestamp('2017-05-05 00:00:00'),
 Timestamp('2017-05-12 00:00:00'),
 Timestamp('2017-05-19 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-06-02 00:00:00'),
 Timestamp('2017-06-09 00:00:00'),
 Timestamp('2017-06-16 00:00:00'),
 Timestamp('2017-06-23 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-07-07 00:00:00'),
 Timestamp('2017-07-14 00:00:00'),
 Timestamp('2017-07-

In [30]:
control_df['timedesc'].unique()

<DatetimeArray>
['2017-02-01 00:00:00', '2017-03-01 00:00:00', '2017-04-01 00:00:00',
 '2017-05-01 00:00:00', '2017-06-01 00:00:00', '2017-07-01 00:00:00',
 '2017-08-01 00:00:00', '2017-09-01 00:00:00', '2017-10-01 00:00:00',
 '2017-11-01 00:00:00', '2017-12-01 00:00:00', '2018-01-01 00:00:00',
 '2018-02-01 00:00:00', '2018-03-01 00:00:00', '2018-04-01 00:00:00',
 '2018-05-01 00:00:00', '2018-06-01 00:00:00', '2018-07-01 00:00:00',
 '2018-08-01 00:00:00', '2018-09-01 00:00:00', '2018-10-01 00:00:00',
 '2018-11-01 00:00:00', '2018-12-01 00:00:00', '2019-01-01 00:00:00',
 '2019-02-01 00:00:00', '2019-03-01 00:00:00', '2019-04-01 00:00:00',
 '2019-05-01 00:00:00', '2019-06-01 00:00:00', '2019-07-01 00:00:00',
 '2019-08-01 00:00:00', '2019-09-01 00:00:00', '2019-10-01 00:00:00',
 '2019-11-01 00:00:00', '2019-12-01 00:00:00', '2020-01-01 00:00:00',
 '2020-02-01 00:00:00', '2020-03-01 00:00:00', '2020-04-01 00:00:00',
 '2020-05-01 00:00:00', '2020-06-01 00:00:00', '2020-07-01 00:00:00',
 '20

In [30]:
media_mrd

brand_vehicle,week_date,CDC+DIGITAL,CDC+TV,EQUITY+DIGITAL,EQUITY+IN STORE,EQUITY+OOH,EQUITY+OTHERADVERTISING_NEWSPAPER_RADIO,EQUITY+TV,PANJAVED+DIGITAL,PANJAVED+IN STORE,PANJAVED+OOH,PANJAVED+TV,SALT+DIGITAL,SALT+IN STORE,SALT+OTHERADVERTISING_NEWSPAPER_RADIO,SALT+TV,TOTAL+DIGITAL,TOTAL+IN STORE,TOTAL+OTHERADVERTISING_NEWSPAPER_RADIO,TOTAL+TV
0,2017-01-06,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0
1,2017-01-13,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0
2,2017-01-20,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0
3,2017-01-27,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0
4,2017-02-03,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2020-11-27,0.0,0.0,1.106151e+06,0.0,0.0,422400.0,2437000.0,625098.632734,0.0,0.0,2665412.5,13.732400,0.0,275000.0,970825.00,864622.889764,1201645.0,0.0,4208625.0
204,2020-12-04,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0
205,2020-12-11,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0
206,2020-12-18,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0


In [31]:
# group control data on timedesc basis
rev_df = control_df.groupby('timedesc')[['volume','value']].sum().reset_index()

In [32]:
# Step 1: Add Year-Month keys to media data
media_mrd['YearMonth'] = media_mrd['week_date'].dt.to_period('M')

In [35]:
media_mrd

brand_vehicle,week_date,CDC+DIGITAL,CDC+TV,EQUITY+DIGITAL,EQUITY+IN STORE,EQUITY+OOH,EQUITY+OTHERADVERTISING_NEWSPAPER_RADIO,EQUITY+TV,PANJAVED+DIGITAL,PANJAVED+IN STORE,PANJAVED+OOH,PANJAVED+TV,SALT+DIGITAL,SALT+IN STORE,SALT+OTHERADVERTISING_NEWSPAPER_RADIO,SALT+TV,TOTAL+DIGITAL,TOTAL+IN STORE,TOTAL+OTHERADVERTISING_NEWSPAPER_RADIO,TOTAL+TV,YearMonth
0,2017-01-06,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0,2017-01
1,2017-01-13,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0,2017-01
2,2017-01-20,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0,2017-01
3,2017-01-27,0.0,1319312.5,1.291382e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,93.177500,0.0,0.0,1780500.00,160.965000,0.0,0.0,0.0,2017-01
4,2017-02-03,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0,2017-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2020-11-27,0.0,0.0,1.106151e+06,0.0,0.0,422400.0,2437000.0,625098.632734,0.0,0.0,2665412.5,13.732400,0.0,275000.0,970825.00,864622.889764,1201645.0,0.0,4208625.0,2020-11
204,2020-12-04,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12
205,2020-12-11,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12
206,2020-12-18,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12


In [33]:
# Step 1: Add Year-Month keys to control data
rev_df['YearMonth'] = rev_df['timedesc'].dt.to_period('M')

In [34]:
rev_df

,timedesc,volume,value,YearMonth
0,2017-02-01,231705.0,99272300.0,2017-02
1,2017-03-01,236111.1,103393400.0,2017-03
2,2017-04-01,237793.8,102741100.0,2017-04
3,2017-05-01,259145.6,107253900.0,2017-05
4,2017-06-01,228778.8,99121000.0,2017-06
5,2017-07-01,248382.3,105892900.0,2017-07
6,2017-08-01,248428.4,105857900.0,2017-08
7,2017-09-01,222033.4,97920700.0,2017-09
8,2017-10-01,228523.8,100496700.0,2017-10
9,2017-11-01,232973.3,99113400.0,2017-11


In [36]:
weeks_in_month = media_mrd.groupby('YearMonth').size().rename('weeks_in_month')

In [37]:
weeks_in_month.shape

(48,)

In [38]:
control_df['timedesc'].unique()

<DatetimeArray>
['2017-02-01 00:00:00', '2017-03-01 00:00:00', '2017-04-01 00:00:00',
 '2017-05-01 00:00:00', '2017-06-01 00:00:00', '2017-07-01 00:00:00',
 '2017-08-01 00:00:00', '2017-09-01 00:00:00', '2017-10-01 00:00:00',
 '2017-11-01 00:00:00', '2017-12-01 00:00:00', '2018-01-01 00:00:00',
 '2018-02-01 00:00:00', '2018-03-01 00:00:00', '2018-04-01 00:00:00',
 '2018-05-01 00:00:00', '2018-06-01 00:00:00', '2018-07-01 00:00:00',
 '2018-08-01 00:00:00', '2018-09-01 00:00:00', '2018-10-01 00:00:00',
 '2018-11-01 00:00:00', '2018-12-01 00:00:00', '2019-01-01 00:00:00',
 '2019-02-01 00:00:00', '2019-03-01 00:00:00', '2019-04-01 00:00:00',
 '2019-05-01 00:00:00', '2019-06-01 00:00:00', '2019-07-01 00:00:00',
 '2019-08-01 00:00:00', '2019-09-01 00:00:00', '2019-10-01 00:00:00',
 '2019-11-01 00:00:00', '2019-12-01 00:00:00', '2020-01-01 00:00:00',
 '2020-02-01 00:00:00', '2020-03-01 00:00:00', '2020-04-01 00:00:00',
 '2020-05-01 00:00:00', '2020-06-01 00:00:00', '2020-07-01 00:00:00',
 '20

In [ ]:
# left join both media and control(only YearMonth and value cols) on YearMonth column
input_df = media_mrd.merge(rev_df[['YearMonth', 'value']], on='YearMonth', how='left')

In [40]:
# Step 4: Merge weeks_in_month
input_df = input_df.merge(weeks_in_month, on="YearMonth", how="left")

In [41]:
# Step 5: Divide revenue equally
input_df["weekly_revenue"] = input_df["value"] / input_df["weeks_in_month"]

In [42]:
print(input_df)

     week_date  CDC+DIGITAL     CDC+TV  EQUITY+DIGITAL  EQUITY+IN STORE  \
0   2017-01-06          0.0  1319312.5    1.291382e+05              0.0   
1   2017-01-13          0.0  1319312.5    1.291382e+05              0.0   
2   2017-01-20          0.0  1319312.5    1.291382e+05              0.0   
3   2017-01-27          0.0  1319312.5    1.291382e+05              0.0   
4   2017-02-03          0.0  1318312.5    1.023217e+05              0.0   
..         ...          ...        ...             ...              ...   
203 2020-11-27          0.0        0.0    1.106151e+06              0.0   
204 2020-12-04          0.0        0.0    4.001368e+05              0.0   
205 2020-12-11          0.0        0.0    4.001368e+05              0.0   
206 2020-12-18          0.0        0.0    4.001368e+05              0.0   
207 2020-12-25          0.0        0.0    4.001368e+05              0.0   

     EQUITY+OOH  EQUITY+OTHERADVERTISING_NEWSPAPER_RADIO  EQUITY+TV  \
0           0.0             

In [43]:
input_df = input_df.fillna(0)

In [44]:
input_df = input_df[input_df['weekly_revenue'] > 0]

In [45]:
input_df

,week_date,CDC+DIGITAL,CDC+TV,EQUITY+DIGITAL,EQUITY+IN STORE,EQUITY+OOH,EQUITY+OTHERADVERTISING_NEWSPAPER_RADIO,EQUITY+TV,PANJAVED+DIGITAL,PANJAVED+IN STORE,PANJAVED+OOH,PANJAVED+TV,SALT+DIGITAL,SALT+IN STORE,SALT+OTHERADVERTISING_NEWSPAPER_RADIO,SALT+TV,TOTAL+DIGITAL,TOTAL+IN STORE,TOTAL+OTHERADVERTISING_NEWSPAPER_RADIO,TOTAL+TV,YearMonth,value,weeks_in_month,weekly_revenue
4,2017-02-03,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0,2017-02,99272300.0,4,24818075.0
5,2017-02-10,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0,2017-02,99272300.0,4,24818075.0
6,2017-02-17,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0,2017-02,99272300.0,4,24818075.0
7,2017-02-24,0.0,1318312.5,1.023217e+05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,85103.892500,0.0,101800.0,2410031.25,158234.765000,200050.0,0.0,1609500.0,2017-02,99272300.0,4,24818075.0
8,2017-03-03,0.0,1066450.0,2.815029e+05,0.0,0.0,0.0,266200.0,0.000000,0.0,0.0,0.0,108540.166000,0.0,61440.0,1673975.00,145203.882000,92080.0,0.0,1101750.0,2017-03,103393400.0,5,20678680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2020-11-27,0.0,0.0,1.106151e+06,0.0,0.0,422400.0,2437000.0,625098.632734,0.0,0.0,2665412.5,13.732400,0.0,275000.0,970825.00,864622.889764,1201645.0,0.0,4208625.0,2020-11,88823900.0,4,22205975.0
204,2020-12-04,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12,90021600.0,4,22505400.0
205,2020-12-11,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12,90021600.0,4,22505400.0
206,2020-12-18,0.0,0.0,4.001368e+05,0.0,0.0,0.0,1620562.5,724498.561816,0.0,555388.5,2960887.5,7.724475,0.0,275000.0,0.00,397030.407948,0.0,0.0,3087600.0,2020-12,90021600.0,4,22505400.0


In [46]:
control_df.columns

Index(['timedesc', 'year', 'month', 'qaurter', 'channel', 'channel_yr',
       'channel_mo', 'channel_qr', 'channeldesc', 'prodhierarchy1',
       'prodhierarchy2', 'proddesc', 'regiondesc', 'region_year', 'value',
       'volume', 'nd', 'wd', 'channel_year', 'prod_year', 'aided_awareness',
       'closeness_top_3_box', 'loyalty', 'top_of_mind_brand_awareness',
       'total_unaided_brand_awareness', 'chim_shop_chai_',
       'panic_pantry_loading', 'welfare', 'welfare_additional_boost',
       'bigpack', 'buyget', 'extravol', 'free_promo', 'free_with_others',
       'freepremium', 'near_pack', 'pmd', 'rl', 'theme', 'tpr', 'target_price',
       'target_baseprice', 'target_discount', 'darlie_discount',
       'darlie_cdc_basepriceratio', 'week5_si', 'week13_si', 'trendline',
       'consumer_confidence_index', 'tourist_arrivals_in_million',
       'extravol_darlie', 'free_promo_darlie', 'free_with_others_darlie',
       'tpr_darlie'],
      dtype='object')

In [48]:
control_var_mapping = pd.read_excel('data_analysis.xlsx', sheet_name='control_glossary')

In [49]:
control_var_mapping = control_var_mapping[['columns','Category']].drop_duplicates()

In [50]:
control_var_mapping = control_var_mapping[~control_var_mapping['Category'].isna()]

In [51]:
control_var_mapping['columns'] = control_var_mapping['columns'].str.lower()

In [52]:
control_var_mapping

,columns,Category
30,nd,Distribution
31,wd,Distribution
32,aided_awareness,Brand Equity
33,closeness_top_3_box,Brand Equity
34,loyalty,Brand Equity
35,top_of_mind_brand_awareness,Brand Equity
36,total_unaided_brand_awareness,Brand Equity
37,chim_shop_chai,Brand Equity
38,panic_pantry_loading,Brand Equity
39,welfare,Promotions & Discounts


{'aided_awareness',
 'bigpack',
 'buyget',
 'chim_shop_chai ',
 'closeness_top_3_box',
 'consumer_confidence_index',
 'darlie_cdc_basepriceratio',
 'darlie_discount',
 'extravol',
 'extravol_darlie',
 'free_promo',
 'free_promo_darlie',
 'free_with_others',
 'free_with_others_darlie',
 'freepremium',
 'loyalty',
 'nd',
 'near pack',
 'panic_pantry_loading',
 'pmd',
 'rl',
 'target_baseprice',
 'target_discount',
 'target_price',
 'theme',
 'top_of_mind_brand_awareness',
 'total_unaided_brand_awareness',
 'tourist_arrivals_in_million',
 'tpr',
 'tpr_darlie',
 'trendline',
 'wd',
 'week13_si',
 'week5_si',
 'welfare',
 'welfare_additional_boost'}

In [ ]:
# correct field names
control_df = control_df.rename(columns={'chim_shop_chai_':'chim_shop_chai'})

In [54]:
correct_dict = {'chim_shop_chai ': 'chim_shop_chai', 'near pack': 'near_pack'}
control_var_mapping['columns'] = control_var_mapping['columns'].apply(lambda x: correct_dict[x] if x in correct_dict else x)

In [57]:
control_var_mapping['columns'].shape

(36,)

In [58]:
control_df.columns.shape

(54,)

In [ ]:
# diff of columns between 2 sets
set(control_var_mapping['columns']) - set(control_df.columns)

In [ ]:
# convert YearMonth into month level
control_df['YearMonth'] = control_df['timedesc'].dt.to_period('M')

In [64]:
req_cols = list(set(control_var_mapping['columns'])) + ['timedesc','YearMonth']

In [65]:
control_df1 = control_df[req_cols]

In [66]:
control_df1

,buyget,panic_pantry_loading,chim_shop_chai,rl,welfare_additional_boost,tourist_arrivals_in_million,tpr_darlie,trendline,pmd,aided_awareness,extravol_darlie,free_with_others_darlie,loyalty,darlie_cdc_basepriceratio,top_of_mind_brand_awareness,near_pack,extravol,nd,free_with_others,target_price,freepremium,consumer_confidence_index,free_promo,week5_si,closeness_top_3_box,bigpack,wd,free_promo_darlie,total_unaided_brand_awareness,target_baseprice,theme,darlie_discount,welfare,target_discount,week13_si,tpr,timedesc,YearMonth
5,0.0,0,0,0.0,0,9.195,0.231079,-22.5,0.0,0.99,0.000000,0.000000,0.00,1.061270,0.00,0.0,0.000000,58.841980,0.000000,468.556058,0.000000,75.8,0.000000,-0.012927,0.0,0.0,64.265460,0.000071,0.00,468.556058,0.0,0.000000,0,0.00000,-0.134394,0.000000,2017-02-01,2017-02
6,0.0,0,0,0.0,0,9.195,0.180081,-22.5,0.0,0.99,0.001674,0.021114,0.00,1.018701,0.00,0.0,0.000000,78.202964,0.000000,400.558692,0.000000,75.8,0.000000,-0.012927,0.0,0.0,77.779744,0.083862,0.00,400.558692,0.0,0.000000,0,0.00000,-0.134394,0.000000,2017-02-01,2017-02
7,0.0,0,0,0.0,0,9.195,0.220098,-22.5,0.0,0.99,0.000000,0.000000,0.00,1.026543,0.00,0.0,0.000000,51.653585,0.000000,367.172152,0.000000,75.8,0.000000,-0.012927,0.0,0.0,82.017702,0.006291,0.00,401.105870,0.0,9.259575,0,8.46004,-0.134394,0.594804,2017-02-01,2017-02
8,0.0,0,0,0.0,0,9.195,0.235401,-22.5,0.0,0.99,0.000000,0.000000,0.00,1.067775,0.00,0.0,0.301696,20.252780,0.005091,449.406432,0.000000,75.8,0.111313,-0.012927,0.0,0.0,26.178953,0.004281,0.00,449.406432,0.0,0.000000,0,0.00000,-0.134394,0.000000,2017-02-01,2017-02
9,0.0,0,0,0.0,0,9.195,0.235401,-22.5,0.0,0.99,0.000000,0.000000,0.00,1.067775,0.00,0.0,0.301696,20.252780,0.005091,449.406432,0.000000,75.8,0.111313,-0.012927,0.0,0.0,26.178953,0.004281,0.00,449.406432,0.0,0.000000,0,0.00000,-0.134394,0.000000,2017-02-01,2017-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,0,0,0.0,0,0.011,0.000000,23.5,0.0,0.99,0.000000,0.000000,0.15,0.956485,0.13,0.0,0.000000,53.703309,0.000000,530.834341,0.000000,50.1,0.000000,-0.128167,0.0,0.0,59.424894,0.000000,0.38,530.834341,0.0,7.264459,0,0.00000,-0.084328,0.000000,2020-12-01,2020-12
236,0.0,0,0,0.0,0,0.011,0.000000,23.5,0.0,0.99,0.005087,0.000000,0.15,1.040735,0.13,0.0,0.000000,54.459074,0.000000,420.766619,0.000000,50.1,0.000000,-0.128167,0.0,0.0,54.822873,0.000000,0.38,420.766619,0.0,0.000000,0,0.00000,-0.084328,0.212761,2020-12-01,2020-12
237,0.0,0,0,0.0,0,0.011,0.000000,23.5,0.0,0.99,0.000000,0.000000,0.15,0.892381,0.13,0.0,0.000000,50.000058,0.000000,454.430257,0.010931,50.1,0.000000,-0.128167,0.0,0.0,74.652132,0.000000,0.38,454.430257,0.0,13.073070,0,0.00000,-0.084328,0.054656,2020-12-01,2020-12
238,0.0,0,0,0.0,0,0.011,0.009730,23.5,0.0,0.99,0.000000,0.000000,0.15,1.052084,0.13,0.0,0.000000,15.606717,0.000350,487.531743,0.000000,50.1,0.229538,-0.128167,0.0,0.0,22.849758,0.000000,0.38,487.531743,0.0,0.000000,0,0.00000,-0.084328,0.000000,2020-12-01,2020-12


In [67]:
control_vars = list(set(control_var_mapping['columns']))

In [68]:
control_vars

['buyget',
 'panic_pantry_loading',
 'chim_shop_chai',
 'rl',
 'welfare_additional_boost',
 'tourist_arrivals_in_million',
 'tpr_darlie',
 'trendline',
 'pmd',
 'aided_awareness',
 'extravol_darlie',
 'free_with_others_darlie',
 'loyalty',
 'darlie_cdc_basepriceratio',
 'top_of_mind_brand_awareness',
 'near_pack',
 'extravol',
 'nd',
 'free_with_others',
 'target_price',
 'freepremium',
 'consumer_confidence_index',
 'free_promo',
 'week5_si',
 'closeness_top_3_box',
 'bigpack',
 'wd',
 'free_promo_darlie',
 'total_unaided_brand_awareness',
 'target_baseprice',
 'theme',
 'darlie_discount',
 'welfare',
 'target_discount',
 'week13_si',
 'tpr']

In [ ]:
# control vars are aggregated on monthly level
control_df1 = control_df1.groupby('YearMonth')[control_vars].mean().reset_index()

In [70]:
control_df1

,YearMonth,buyget,panic_pantry_loading,chim_shop_chai,rl,welfare_additional_boost,tourist_arrivals_in_million,tpr_darlie,trendline,pmd,aided_awareness,extravol_darlie,free_with_others_darlie,loyalty,darlie_cdc_basepriceratio,top_of_mind_brand_awareness,near_pack,extravol,nd,free_with_others,target_price,freepremium,consumer_confidence_index,free_promo,week5_si,closeness_top_3_box,bigpack,wd,free_promo_darlie,total_unaided_brand_awareness,target_baseprice,theme,darlie_discount,welfare,target_discount,week13_si,tpr
0,2017-02,0.000000,0.0,0.0,0.000000,0.0,9.195,0.220412,-22.5,0.000000,0.990000,0.000335,0.004223,0.000000,1.048413,0.00,0.000000,0.120678,45.840818,0.002036,427.019953,0.000000,75.8,0.044525,-0.012927,0.000000,0.000000,55.284163,0.019757,0.000000,433.806697,0.000000,1.851915,0.0,1.692008,-0.134394,0.118961
1,2017-03,0.000000,0.0,0.0,0.000000,0.0,9.195,0.199166,-21.5,0.000000,0.990000,0.001551,0.005148,0.170000,1.050531,0.27,0.000000,0.139663,45.120969,0.001490,432.009081,0.000000,76.8,0.042974,0.242712,0.000000,0.000000,54.985101,0.026854,0.470000,434.297166,0.000000,2.737506,0.0,0.000000,-0.009762,0.123373
2,2017-04,0.000000,0.0,0.0,0.000000,0.0,8.130,0.209780,-20.5,0.000000,1.000000,0.000054,0.006498,0.130000,1.033367,0.25,0.000000,0.026516,45.035293,0.000696,427.848249,0.000000,77.0,0.042930,0.092988,0.000000,0.000000,55.259852,0.027630,0.400000,434.297166,0.000000,1.927634,0.0,0.000000,0.086798,0.112624
3,2017-05,0.046572,0.0,0.0,0.000000,0.0,8.130,0.219881,-19.5,0.000000,1.000000,0.000048,0.007079,0.160000,1.050255,0.26,0.000000,0.029748,45.605957,0.000941,417.600502,0.000000,76.0,0.042224,-0.076841,0.460000,0.000000,55.669223,0.022314,0.410000,432.235162,0.000000,0.000000,0.0,2.941260,0.099561,0.162431
4,2017-06,0.000000,0.0,0.0,0.000000,0.0,8.130,0.229809,-18.5,0.000000,1.000000,0.000028,0.002098,0.150000,1.061324,0.25,0.000000,0.035583,45.495242,0.000382,426.942635,0.000000,74.9,0.036176,0.275774,0.540000,0.000000,55.790871,0.008477,0.430000,430.621262,0.000000,0.000000,0.0,0.000000,0.088301,0.078279
5,2017-07,0.000000,0.0,0.0,0.000000,0.0,8.820,0.207541,-17.5,0.000000,1.000000,0.000030,0.002208,0.160000,1.067728,0.27,0.000000,0.043102,44.389336,0.000308,421.747842,0.000000,73.9,0.040462,-0.025510,0.480000,0.000000,54.833804,0.013501,0.420000,428.469574,0.000000,0.000000,0.0,1.105270,0.088196,0.117298
6,2017-08,0.063640,0.0,0.0,0.000000,0.0,8.820,0.227304,-16.5,0.000000,1.000000,0.000036,0.000000,0.130000,1.060573,0.24,0.000000,0.046009,44.522800,0.000853,425.374737,0.000000,74.5,0.038967,0.036396,0.000000,0.000000,53.963014,0.005489,0.420000,431.029581,0.000000,1.067501,0.0,1.455389,0.088525,0.123634
7,2017-09,0.000000,0.0,0.0,0.000000,0.0,8.820,0.189822,-15.5,0.000000,1.000000,0.000220,0.000000,0.130000,1.060167,0.30,0.000000,0.035625,45.571588,0.000361,434.113752,0.000000,75.0,0.032109,0.086176,0.000000,0.000000,55.171516,0.003501,0.450000,435.092076,0.000000,0.000000,0.0,0.000000,0.014958,0.120560
8,2017-10,0.000000,0.0,0.0,0.000000,0.0,9.390,0.175202,-14.5,0.000000,1.000000,0.000390,0.000000,0.120000,1.047646,0.25,0.000000,0.033483,44.947378,0.000378,436.107114,0.000000,76.7,0.034076,-0.126212,0.000000,0.000000,54.945981,0.001883,0.450000,438.070340,0.000000,0.000000,0.0,0.000000,-0.001400,0.205402
9,2017-11,0.000000,0.0,0.0,0.000000,0.0,9.390,0.191432,-13.5,0.000000,1.000000,0.000171,0.000000,0.160000,1.052338,0.28,0.000000,0.025882,44.610702,0.000303,424.379789,0.000000,78.0,0.039778,-0.022389,0.000000,0.000000,54.761663,0.000080,0.400000,437.715242,0.000000,1.304409,0.0,2.277314,-0.029949,0.126099


In [71]:
input_df.shape

(204, 24)

In [72]:
input_df1 = input_df.merge(control_df1, on='YearMonth',how='left')

In [73]:
input_df1.shape

(204, 60)

In [ ]:
# add control variables to existing media data - done

# do adstocking - geometric adstock, make a function of this and keep decay rate such that you can easily change it
# do hill adstocking - hill adstock, make a function of this and keep alpha and gamma such that you can easily change it
# normalizing
# getting prior distribution for last year same quarter and previous quarters
# pymc bayesian modeling
# get contribution, ROI, R square, MAPE
# make code modularised and structured (you can save functions in diff py files)
# make pipelines
# make transformations such that you can easily untransform


